In [17]:
import os
import re
import urllib3
from Crypto.Cipher import AES

def fetch(r : str, url : str) -> str:
    http = urllib3.PoolManager()
    result = http.request("GET", url)
    data = result.data.decode("utf-8")
    try:
        return re.search(r, data).group(0)
    except:
        return ""

def decryptAes(secret : bytes, data : bytes) -> bytes :
    aes = AES.new(secret, AES.MODE_ECB)
    decryptedData = aes.decrypt(data)
    return decryptedData

def encryptAes(secret : bytes, data : bytes) -> bytes :
    aes = AES.new(secret, AES.MODE_ECB)
    encryptedData = aes.encrypt(data)
    return encryptedData

def decryptString(secret : bytes, data : str) -> bytes :
    srcBytes = data.encode("utf-8")
    srcBytes = deflateBytes(srcBytes)
    srcBytes = decryptAes(secret=secret, data=srcBytes)
    while srcBytes[-1] == 0:
        srcBytes = srcBytes[ : -1]
    return srcBytes

def encryptString(secret : bytes, data : str) -> bytes :
    srcBytes = data.encode("utf-8")
    if len(srcBytes) % 16 != 0:
        lenOfSrcBytes = len(srcBytes)
        for i in range(16 - (lenOfSrcBytes % 16)):
            srcBytes += " ".encode("utf-8")
    srcBytes = encryptAes(secret=secret, data=srcBytes)
    return inflateBytes(srcBytes)

def inflateBytes(data : bytes) -> bytes :
    result = bytes()
    for i in range(len(data)):
        result += ((data[i] & 240) >> 4).to_bytes(1, byteorder="big", signed=False)
        result += ((data[i] & 15)).to_bytes(1, byteorder="big", signed=False)
    return result

def deflateBytes(data : bytes) -> bytes :
    result = bytes()
    for i in range(len(data)):
        if i % 2 == 0:
            result += ((data[i] << 4) + data[i + 1]).to_bytes(1, byteorder="big", signed=False)
    return result

#################################################################################################################################

def encryptSingleJekyll(url : str, filename : str, srcSecret : bytes, dstSecret : bytes):
    file = open(filename, "r", encoding="utf-8")
    content = file.read()
    file.close()
    frontmatter = re.search(r"---(.|\s|\n)*?---", content).group(0)
    layoutEncrypt = re.search(r"---(.|\s|\n)*?layout\s*?:\s*?encrypt(.|\s|\n)*?---", frontmatter)
    # ! layout: encrypt
    if layoutEncrypt is None:
        # article = fetch(url=url, r=r"<article(.|\s|\n)*?article>")
        article = "" + \
        "123123"
        encryptedArticle = encryptString(secret=dstSecret, data=article).decode("utf-8")
        frontmatter = re.sub(r"layout\s*?:\s*?[A-Za-z0-9_]+", "layout: encrypt", frontmatter)
        content = "" + \
        frontmatter + "\n" + \
        r"`" + encryptedArticle + r"`{:.article}"
        file = open(filename, "w+", encoding="utf-8") 
        file.write(content)
        file.flush()
        file.close()
    # layout: encrypt
    else:
        article = fetch(url=url, r=r"<code[a-zA-Z0-9_=\"\-\s]+article(.|\n|\s)*?code>")
        encryptedArticle = ""
        decryptedArticle = ""
        newEncryptedArticle = ""
        try:
            encryptedArticle = re.search(r">(.|\s|\n)*?<", article).group(0)[1:-1]
            decryptedArticle = decryptString(secret=srcSecret, data=encryptedArticle).decode("utf-8")
            newEncryptedArticle = encryptString(secret=dstSecret, data=decryptedArticle).decode("utf-8")
        except:
            pass
        content = "" + \
        frontmatter + "\n" + \
        r"`" + newEncryptedArticle + r"`{:.article}"
        file = open(filename, "wb+") 
        file.write(content)
        file.flush()
        file.close()

def decryptSingleJekyll(secret : bytes, filename : str):
    file = open(filename, "r", encoding="utf-8")
    content = file.read()
    file.close()
    frontmatter = re.search(r"---(.|\s|\n)*?---", content).group(0)
    frontmatter = re.sub(r"layout\s*?:\s*?[a-zA-Z0-9_]+", "layout: article", frontmatter)
    encryptedArticle = re.search(r"`(.|\s|\n)*?`{:.article}", content).group(0)[1 : -12]
    decryptedArticle = decryptString(secret=secret, data=encryptedArticle).decode("utf-8")
    file = open(filename, "wb+")
    content = frontmatter + "\n" + \
    decryptedArticle
    file.write(content)
    file.flush()
    file.close()

In [18]:
encryptSingleJekyll(url="", filename="./test.html", srcSecret=b"1234123412341234", dstSecret=b"1234123412341234")

In [19]:
decryptSingleJekyll(secret=b"1234123412341234", filename="./test.html")